In [ ]:
from math import sin, cos, pi     #数学関数・定数のインポート
import numpy as np                #数値計算機能のインポート 別名np
import matplotlib.pyplot as plt   #グラフ描画機能のインポート 別名plt
import ipywidgets as widgets      #対話的処理機能のインポート

# 同次変換

### 同次変換行列(2次元)を作る関数

In [ ]:
def H_2d(angle, r):

  H = np.array([
    [ cos(angle), -sin(angle), r[0] ],
    [ sin(angle),  cos(angle), r[1] ],
    [          0,           0,   1  ],
  ])

  return H

In [ ]:
H_2d(pi/2, [1,0]) #お試し

※ `6.123234e-17` $=6.123234\times10^{-17}$なので，計算機誤差を含む 0 のこと．

### ダミー成分を追加・除去する関数

In [ ]:
### 物理座標にダミー成分を追加する関数
def add_dummy(x):
  return np.array([x[0], x[1], 1]) #ダミー成分を追加

### 同次座標からダミー変数を除去する関数
def remove_dummy(xx):
  return np.array([xx[0], xx[1]]) #ダミー成分を除去

In [ ]:
H = H_2d(pi/2, [3,4]) #お試し
x = np.array([1,2])
print(x)
hx = add_dummy(x)
print(hx)
hy = np.dot(H, hx) #同次変換行列と同次座標の積
print(hy)
y = remove_dummy(hy)
print(y)

## 三角形の同次変換

### 三角形のデータ（初期状態）

In [ ]:
points0 = np.array([
  [ 2,  2], #[x,y]座標　1点目
  [-4, -1], #2点目
  [ 2, -1], #3点目
  [ 2,  2], #4点目 = 1点目
])

### 三角形を描く関数

In [ ]:
def plot_points( points ):

  plt.plot( points[:,0], points[:,1] )
  plt.xlabel('x')
  plt.ylabel('y')
  plt.grid()

In [ ]:
plot_points( points0 ) #初期状態をプロット

### 三角形を同次変換する関数

In [ ]:
def H_by_points( points, angle, r ):

  H = H_2d(angle, r)  #同次変換行列
  newpoints = []  #空のリスト
  for point in points:
    hp = add_dummy(point) #同次座標化
    newhp = np.dot(H, hp) #同次変換
    newp = remove_dummy(newhp)
    newpoints.append(newp)

  return np.array(newpoints)

90度回してみる

In [ ]:
newpoints = H_by_points( points0, pi/2, [0,0] ) #回転角π/2, 平行移動(rx=0,ry=0)
print(newpoints)
plot_points(newpoints)

## 手動操作

In [ ]:
def show_widgets():
  '''
  手動で図形を動かす
  '''
  def make_slider(name, val, min, max, step):
    spec = {'value':val, 'min':min, 'max':max, 'step':step,
            'discription':name, 'readout_format':'.1f'}
    return widgets.FloatSlider(**spec)

  rx_input = make_slider('ry:', 0, -10, 10, 1)
  ry_input = make_slider('ry:', 0, -10, 10, 1)
  th_input = make_slider('angle:', 0, -180, 180, 10)
  output = widgets.Output()

  fig, ax = plt.subplots(1,1, figsize=(5,5))   #正方形の用紙

  @output.capture(clear_output=True, wait=True)
  def draw(rx, ry, th):

    th_rad = th/180*pi
    newpoints = H_by_points( points0, th_rad, [rx, ry] )

    ax.clear()
    ax.set_xlim([-10,10])             #x軸の範囲
    ax.set_ylim([-10,10])             #y軸の範囲
    ax.set_xlabel('x')                #x軸のラベル
    ax.set_ylabel('y')                #y軸のラベル
    ax.set_title('r = (%f, %f), angle=%f'%(rx,ry,th))
    ax.plot( newpoints[:,0], newpoints[:,1] )
    ax.grid()

    display(ax.figure)

  draw(0, 0, 0) #初期姿勢
  widgets.interactive(draw, rx=rx_input, ry=ry_input, th=th_input)
  display(rx_input, ry_input, th_input, output)
  plt.close() #余分なプロットを抑制

show_widgets()